# Nama : Peno
# Nim : 221220095
# Kelas : 29 Informatika

In [7]:
from collections import defaultdict
import re
import pandas as pd

df = pd.read_csv('berita_olahraga_scraped.csv')
df.head()


,No,Judul,Isi,Url,Label
0,1,PBSI Hanya Kirim 7 Wakil ke Singapore Open 2025,PP PBSI mengirim skuad minimalis ke turnamen B...,https://sport.detik.com/raket/d-7931755/pbsi-h...,Non-Sepak Bola
1,2,Cabor Bulutangkis Kisruh soal Batasan Umur Atl...,Cabang olahraga (cabor) bulutangkis kisruh men...,https://www.detik.com/bali/berita/d-7922150/ca...,Non-Sepak Bola
2,3,Rekap Final Thailand Open 2025,Malaysia dan China sama-sama mengamankan dua g...,https://sport.detik.com/raket/d-7920224/rekap-...,Non-Sepak Bola
3,4,Jonatan Christie Antara Program Latihan Hingga...,Jonatan Christiemengaku telah mempersiapkan se...,https://sport.detik.com/raket/d-7919041/jonata...,Non-Sepak Bola
4,5,Blak-blakan Jonatan Christie Ungkap Alasan Mun...,Jonatan Christie memutuskan mundur dari Pelatn...,https://www.detik.com/jogja/berita/d-7918868/b...,Non-Sepak Bola


In [8]:
print("📄 Jumlah dokumen:", len(df))

📄 Jumlah dokumen: 352


In [9]:
def tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())
# tokenisasi
df['Tokens'] = df['Isi'].apply(tokenize)
print(df[['Isi', 'Tokens']].head())

                                                 Isi  \
0  PP PBSI mengirim skuad minimalis ke turnamen B...   
1  Cabang olahraga (cabor) bulutangkis kisruh men...   
2  Malaysia dan China sama-sama mengamankan dua g...   
3  Jonatan Christiemengaku telah mempersiapkan se...   
4  Jonatan Christie memutuskan mundur dari Pelatn...   

                                              Tokens  
0  [pp, pbsi, mengirim, skuad, minimalis, ke, tur...  
1  [cabang, olahraga, cabor, bulutangkis, kisruh,...  
2  [malaysia, dan, china, sama, sama, mengamankan...  
3  [jonatan, christiemengaku, telah, mempersiapka...  
4  [jonatan, christie, memutuskan, mundur, dari, ...  


In [10]:
def build_inverted_index(docs):
    index = defaultdict(set)
    for doc_id, text in docs.items():
        for word in tokenize(text):
            index[word].add(doc_id)
    return index

dokumen = df['Isi'].to_dict()
inverted_index = build_inverted_index(dokumen)
universal_set = set(dokumen.keys())
# Tampilkan inverted index
for i, (term, posting) in enumerate(sorted(inverted_index.items())):
    print(f"{term:15} → {sorted(posting)}")

0               → [9, 27, 30, 31, 32, 33, 35, 36, 37, 62, 79, 92, 116, 119, 123, 124, 125, 127, 131, 133, 134, 135, 136, 137, 139, 142, 145, 147, 148, 149, 151, 152, 153, 158, 163, 169, 172, 192, 197, 198, 200, 202, 203, 204, 205, 206, 207, 227, 229, 231, 234, 235, 236, 242, 251, 266, 274, 275, 276, 278, 284, 289, 290, 292, 297, 299, 310, 320, 321, 328, 335, 337, 339, 341, 345, 346, 347, 349, 350, 351]
00              → [20, 29, 43, 48, 52, 94, 95, 121, 123, 124, 127, 128, 129, 130, 131, 132, 135, 136, 137, 138, 140, 141, 142, 143, 169, 196, 220, 222, 223, 224, 232, 233, 236, 238, 252, 313]
000             → [28, 52, 68, 87, 92, 209, 259, 289, 310, 311, 345]
01              → [142, 169]
02              → [236, 290]
03              → [16, 169, 186, 310]
04              → [173, 174, 236, 290]
05              → [25, 35, 204, 205, 236, 249, 290, 346]
06              → [35, 50, 173, 174, 186, 204, 205, 310]
08              → [29, 37, 48, 94, 95, 220, 223, 224, 233]
09              → [25, 48

In [11]:
def boolean_search(query):
    tokens = tokenize(query)
    if not tokens:
        return set()
    
    def term(pos):
        neg = False
        if tokens[pos] == "not":
            neg = True
            pos += 1
            if pos >= len(tokens):
                raise ValueError("Operator NOT tidak diikuti term.")
        word = tokens[pos]
        postings = set(inverted_index.get(word, set()))
        if neg:
            postings = universal_set - postings
        return postings, pos + 1
    result, pos = term(0)
    while pos < len(tokens):
        op = tokens[pos]
        if op not in ("and", "or"):
            raise ValueError(f"Operator tidak dikenali: {op}")
        pos += 1
        rhs, pos = term(pos)
        if op == "and":
            result &= rhs
        else:
            result |= rhs
    return result

results = {}


In [12]:
queries = [
    "Anthony and MU or psg and menang",
    "Badminton or Basket",
    "Laliga and not Messi",
    "dari and not mesin",
    "not Inggris",
    "Milan and Kalah"
]
for q in queries:
    try:
        hasil = boolean_search(q)
        results[q] = sorted(hasil)
        print(f"Query: '{q}' → {sorted(hasil)}")
    except (IndexError, ValueError) as e:
        results[q] = sorted(hasil)
        print(f"Query: '{q}' → Error: {e}")

Query: 'Anthony and MU or psg and menang' → [170, 192, 292]
Query: 'Badminton or Basket' → [3, 23, 30, 37, 39, 41, 42, 43, 44, 47, 48, 49, 50, 51, 52, 55, 60, 65, 67, 69, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 268]
Query: 'Laliga and not Messi' → [162, 251, 253, 255, 289, 291, 295, 296, 298, 303, 304, 305, 317]
Query: 'dari and not mesin' → [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 118, 119, 120, 122, 125, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,